In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import ResidualBlock, Discriminator
import torchvision
import torch

image = torchvision.transforms.Resize(256)(torchvision.io.read_image("face.jpg").float()).unsqueeze(0)

In [3]:
image.size()

torch.Size([1, 3, 256, 256])

In [4]:
rb = ResidualBlock(3, 16)

In [5]:
out = rb(image)
out.size()

torch.Size([1, 16, 128, 128])

In [6]:
d = Discriminator(7, 16)

In [7]:
print(image.size())
d(image)

torch.Size([1, 3, 256, 256])


tensor([[0.4998]], grad_fn=<SigmoidBackward0>)

In [8]:
test = torch.zeros((8,3,256,256))
d(test).size()

torch.Size([8, 1])

In [9]:
import torch
torch.manual_seed(9)
#layer = torch.nn.Linear(1, 4)
# layer.weight *= 4 inplace operations on leaf variables does not work

con_layer = torch.nn.Conv2d(4,16, kernel_size=3)
test = torch.zeros((8,4,256,256))
test_unf = torch.nn.functional.unfold(test, (3, 3))
weight = torch.randn(size=(16, 4, 3, 3))
a = test_unf.transpose(1, 2).matmul(weight.view(weight.size(0), -1).t())
print(a.size())
print(con_layer.weight.size(), con_layer(test).size(), test.size(), test_unf.size())

torch.Size([8, 64516, 16])
torch.Size([16, 4, 3, 3]) torch.Size([8, 16, 254, 254]) torch.Size([8, 4, 256, 256]) torch.Size([8, 36, 64516])


In [10]:
import torch
from torch import nn
unfold = nn.Unfold(kernel_size=(2, 3))
input = torch.randn(2, 5, 3, 4)
output = unfold(input)
# each patch contains 30 values (2x3=6 vectors, each of 5 channels)
# 4 blocks (2x3 kernels) in total in the 3x4 input
# print(input.size(),output.size())

# Convolution is equivalent with Unfold + Matrix Multiplication + Fold (or view to output shape)
inp = torch.randn(4, 3, 10, 12)
w = torch.randn(2, 3, 4, 5)
inp_unf = torch.nn.functional.unfold(inp, (4, 5))
print(inp_unf.size())
out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
print(out_unf.size())
out = torch.nn.functional.fold(out_unf, (7, 8), (1, 1))
print(out.size())
# or equivalently (and avoiding a copy),
# out = out_unf.view(1, 2, 7, 8)
(torch.nn.functional.conv2d(inp, w) - out).abs().max()

torch.Size([4, 60, 56])
torch.Size([4, 2, 56])
torch.Size([4, 2, 7, 8])


tensor(5.7220e-06)

In [11]:
w.view(w.size(0), -1).t().size()

torch.Size([60, 2])

In [12]:
torch.manual_seed(9)
def convolution(i, w):

    inp_unf = torch.nn.functional.unfold(i, (w.size(2), w.size(3)))
    if len(w.size()) == 4:
        out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), -1).t()).transpose(1, 2)
    else:
        print("heuy", inp_unf.transpose(1, 2).size(), w.view(w.size(0), w.size(1), -1).transpose(1,2).size())
        out_unf = inp_unf.transpose(1, 2).matmul(w.view(w.size(0), w.size(1), -1).transpose(1,2)).transpose(1, 2)
    out = torch.nn.functional.fold(out_unf, (i.size(2)-w.size(2)+1, i.size(3)-w.size(3)+1), (1, 1))
    print(out.size())
    return out

In [13]:
torch.manual_seed(9)
i = torch.ones((8,3,6,6))
s2 = torch.rand(8,1,3,1,1)
s1 = s2.view(8,3,1,1)
# print(s)
w = torch.ones((16, 3, 3, 3))
o1 = convolution(i*s1, w)
bias = torch.randn((16,1,1))
print(o1+bias)
print((s2*w).size())
o2 = convolution(i, s2*w)
print((~(torch.abs(o1-o2) <1e-3)).sum())

torch.Size([8, 16, 4, 4])
tensor([[[[13.7770, 13.7770, 13.7770, 13.7770],
          [13.7770, 13.7770, 13.7770, 13.7770],
          [13.7770, 13.7770, 13.7770, 13.7770],
          [13.7770, 13.7770, 13.7770, 13.7770]],

         [[13.0996, 13.0996, 13.0996, 13.0996],
          [13.0996, 13.0996, 13.0996, 13.0996],
          [13.0996, 13.0996, 13.0996, 13.0996],
          [13.0996, 13.0996, 13.0996, 13.0996]],

         [[12.6039, 12.6039, 12.6039, 12.6039],
          [12.6039, 12.6039, 12.6039, 12.6039],
          [12.6039, 12.6039, 12.6039, 12.6039],
          [12.6039, 12.6039, 12.6039, 12.6039]],

         ...,

         [[12.2737, 12.2737, 12.2737, 12.2737],
          [12.2737, 12.2737, 12.2737, 12.2737],
          [12.2737, 12.2737, 12.2737, 12.2737],
          [12.2737, 12.2737, 12.2737, 12.2737]],

         [[11.6748, 11.6748, 11.6748, 11.6748],
          [11.6748, 11.6748, 11.6748, 11.6748],
          [11.6748, 11.6748, 11.6748, 11.6748],
          [11.6748, 11.6748, 11.6748, 1

In [14]:
a = torch.randn((8,3,2,1,1))
a.square().sum(dim=(2,3,4), keepdim=True).size()

torch.Size([8, 3, 1, 1, 1])

In [15]:
a,a.square().sum(dim=(2,3,4))

(tensor([[[[[ 0.3581]],
 
           [[ 0.5669]]],
 
 
          [[[ 0.0802]],
 
           [[-1.1982]]],
 
 
          [[[-0.0381]],
 
           [[-0.0890]]]],
 
 
 
         [[[[-0.6173]],
 
           [[-0.6526]]],
 
 
          [[[-2.3320]],
 
           [[ 0.1985]]],
 
 
          [[[-1.7287]],
 
           [[-0.6561]]]],
 
 
 
         [[[[-0.6030]],
 
           [[-1.6146]]],
 
 
          [[[ 0.3054]],
 
           [[ 1.8163]]],
 
 
          [[[-0.9805]],
 
           [[ 0.8611]]]],
 
 
 
         [[[[-0.3283]],
 
           [[-1.3671]]],
 
 
          [[[ 0.0633]],
 
           [[ 1.2585]]],
 
 
          [[[-0.8138]],
 
           [[-1.1117]]]],
 
 
 
         [[[[-0.2294]],
 
           [[ 0.1999]]],
 
 
          [[[ 0.0027]],
 
           [[ 0.6918]]],
 
 
          [[[ 0.8121]],
 
           [[-0.6082]]]],
 
 
 
         [[[[-0.5209]],
 
           [[ 1.6300]]],
 
 
          [[[-0.0349]],
 
           [[-1.2214]]],
 
 
          [[[ 0.4809]],
 
           [[ 1.2575]]]]

In [16]:
from model import ModDemodConv3x3
mdconv = ModDemodConv3x3(3, 16)
i = torch.ones((8,3,256,256))
s = torch.rand(8,3)
o=mdconv(i,s)
o.size()

torch.Size([8, 16, 256, 256])

In [17]:
input = torch.arange(1, 5, dtype=torch.float32).view(1, 1, 2, 2)
print(input.size())

m = torch.nn.Upsample(scale_factor=2, mode='nearest')
print(m(input).size())

m = torch.nn.Upsample(scale_factor=2, mode='bilinear')  
print(m(input).size())

torch.Size([1, 1, 2, 2])
torch.Size([1, 1, 4, 4])
torch.Size([1, 1, 4, 4])


In [18]:
x = torch.tensor([[1], [2], [3]])
print(x.size())
print(x.repeat(1,4).size())   # -1 means not changing the size of that dimension
print(x, x.repeat(1,4))

torch.Size([3, 1])
torch.Size([3, 4])
tensor([[1],
        [2],
        [3]]) tensor([[1, 1, 1, 1],
        [2, 2, 2, 2],
        [3, 3, 3, 3]])


In [19]:
x.unsqueeze(0).size()

torch.Size([1, 3, 1])

In [20]:
torch.zeros([]).size()

torch.Size([])

In [21]:
noise_scaling_factor = torch.tensor([2])
noise = torch.randn((1,1,4,4))

In [22]:
noise, noise_scaling_factor * noise, noise_scaling_factor.size()

(tensor([[[[-0.8339,  0.0774, -0.1816, -1.2314],
           [-1.9383,  1.8579,  1.1635, -0.3830],
           [-0.7162,  0.1926, -0.0437,  1.2684],
           [-0.5551, -0.1982,  2.1788,  0.2799]]]]),
 tensor([[[[-1.6678,  0.1548, -0.3631, -2.4628],
           [-3.8767,  3.7157,  2.3270, -0.7661],
           [-1.4324,  0.3852, -0.0874,  2.5368],
           [-1.1103, -0.3964,  4.3576,  0.5599]]]]),
 torch.Size([1]))

In [23]:
torch.zeros((1))*noise

tensor([[[[-0., 0., -0., -0.],
          [-0., 0., 0., -0.],
          [-0., 0., -0., 0.],
          [-0., -0., 0., 0.]]]])

In [24]:
upsample = torch.nn.Upsample(scale_factor=2, mode="bilinear")
inp = torch.randn(8,3,4,4)
inp.size(), upsample(inp).size()

(torch.Size([8, 3, 4, 4]), torch.Size([8, 3, 8, 8]))

In [25]:
layer = torch.nn.Conv2d(3,16,3)
layer.weight.size(), layer.bias.size()

(torch.Size([16, 3, 3, 3]), torch.Size([16]))

In [26]:
inp = torch.randn((8,3,16,16))

In [27]:
layer(inp).size()

torch.Size([8, 16, 14, 14])

In [28]:
import torch
from model import HistoGAN
from torchinfo import summary

histogan = HistoGAN()
summary(histogan, [(8, 512), (8, 3, 256, 256)])

torch.Size([8, 512])
torch.Size([8, 64, 4, 4])


RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: [Sequential: 1, Linear: 2, LeakyReLU: 2, Linear: 2, LeakyReLU: 2, Linear: 2, LeakyReLU: 2, Linear: 2, LeakyReLU: 2, Linear: 2, LeakyReLU: 2, Linear: 2, LeakyReLU: 2, Linear: 2, LeakyReLU: 2, Linear: 2, Linear: 3, Linear: 3, Linear: 3, ModDemodConv3x3: 3]

In [ ]:
# from data import AnimeFacesDataset
# from torchvision.transforms import Resize
# resize = Resize((256,256))
# image_dir = "images"
# dataset = AnimeFacesDataset(image_dir)

In [ ]:
# dataset[0].size()

In [38]:
batch = torch.arange(4).view(4, 1)
# batch = batch.repeat(1,4)
# select 5 samples randomly
for i in range(4):
    idx = torch.randperm(4)[:2]
    mini_batch = batch[idx]
    a = torch.rand(1)

In [39]:
batch.size()
idx
mini_batch, batch

(tensor([[3],
         [2]]),
 tensor([[0],
         [1],
         [2],
         [3]]))

In [45]:
a = torch.rand((4,1,1,1))
a, 1-a

(tensor([[[[0.6037]]],
 
 
         [[[0.9881]]],
 
 
         [[[0.9928]]],
 
 
         [[[0.8181]]]]),
 tensor([[[[0.3963]]],
 
 
         [[[0.0119]]],
 
 
         [[[0.0072]]],
 
 
         [[[0.1819]]]]))